### José de Jesús Tapia López
### Procesamiento de Lenguaje Natural
### 18 de Octubre del 2020
### Tema 2 - Actividad Sumativa 4

Identificar correctamente todas las diferentes variantes de fecha codificadas en este conjunto de datos, normalizar y ordenar adecuadamente las fechas. Cada línea del archivo dates.txt corresponde a una nota médica. Cada nota tiene una fecha que debe extraerse, pero cada fecha está codificada en uno de muchos formatos. Aquí hay una lista de algunas de las variantes que puede encontrar en este conjunto de datos:

* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Una vez que haya extraído estos patrones de fecha del texto, el siguiente paso es clasificarlos en orden cronológico ascendente de acuerdo con las siguientes reglas:

* Todas las fechas están en formato xx/xx/ xx son mm/dd/aa
* Todas las fechas en las que el año está codificado en solo dos dígitos corresponden a años posteriores a la década de 1900 (p. Ej., 1/5/89 es el 5 de enero de 1989).
* Si falta el día (p. Ej., 9/2009), suponga que es el primer día del mes (p. Ej., septiembre, 1 de 2009).
* Si falta el mes (por ejemplo, 2010), suponga que es el primero de enero de ese año (p. Ej., enero, 1 de 2010).
* Tenga cuidado con los posibles errores tipográficos, ya que este es un conjunto de datos derivados de la vida real.


Esta función debería devolver una lista de longitud 500.


Adjuntar el código de Python, en un Jupyter Notebook. El código debe estar bien documentado.

In [1]:
import re
import pandas as pd
pd.set_option('display.max_rows', 500)

In [2]:
# Caso 1: 04/20/2009; 04/20/09; 4/20/09; 4/3/09 
# La primera entrada (mm) puede tener de un número de una o dos cifras,
# la segunda entrada (dd) puede tener de un número de una o dos cifras y la tercer entrada 
# puede tener un número de dos a cuatro cifras (aaaa).


#\d Coincide con cualquier dígito decimal; esto es equivalente a la clase [0-9]
# '{n,m}' indica que hay entre n y m repeticiones
# [/-] indica que puede aparacer el símbolo "/" o "-" (para separar mes con el dia y dia con el año)

caso1 = '(\d{1,2}[/-]\d{1,2}[/-]\d{2,4})'

# Caso 2: 6/2008; 12/2009

# Un número de uno o dos dígitos seguido de un símbolo "/" o "-" seguido de un número 1 o un número 2 (Porque
# puede que haya un numero como 3007 pero es fecha aún no ocurre) y eso último seguido de número va a un 
# número de 3 cifras (Para completar los miles, es decir, los números con cuatro cifras)

caso2 = '(\d{1,2}[/-][1|2]\d{3})'

# Caso 3: Mar-20-2009. 
# La primera letra contiene una mayúzcula (Por eso el rango [A-Z]) 
# Después de la mayúscula, puede haber UNA o más letras minúsculas (Por eso el rango [a-z]+)
# Posteriormente, dicho mes puedes estar separado por un '\' o un '-', (Por eso [/-])
# Luego sigue un \d, que coincide con cualquier dígito decimal; esto es equivalente a la clase [0-9]
# '{n,m}' indica que hay entre n y m repeticiones (en este caso un número que tiene una cifra o dos)
# Finalmente están otras cosas que ya expliqué:
# el último puedes estar separado por un '\' o un '-', (Por eso [/-])
# y al último un \d{2,4} para indicar que hay entre dos y cuatro repeticiones de un dígito decimal 

caso3 = '[A-Z][a-z]+[/-]\d{1,2}\[/-]\d{2,4}'

# Caso 4:  Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;

# indicamos que al principio esten las tres primeras palabras de algún mes (por si estan abreviadas) y puede
# que el nombre del mes esté completo, por ello a las tres primeras iniciales del mes pueden seguirles cero o más
# letras en minúsculas (por ello [a-z]*); a ese le puede seguir: una coma, un punto o ninguno de esos dos 
# (por eso [,.]) después le sigue entre dos y cuatro repeticiones de un dígito decimal 

caso4 = "(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[,.]? \d{1,2}[,.]? \d{2,4}"

# Caso 5: 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009, Feb 2009; Sep 2009; Oct 2010

# Puede o no haber un número que tiene entre uno y dos cifras y seguido de un espacio en blanco, 
# seguido de las tres primeras palabras de algún mes (por si estan abreviadas) y puede
# que el nombre del mes esté completo, por ello a las tres primeras iniciales pueden seguirles cero o más
# letras en minúsculas (por ello [a-z]*), seguido de una posible o nula aparición
# de los signos ',' o '.', seguida de un espacio en blanco, seguida de un número de entre dos y cuatro cifras

caso5 = '(?:\d{1,2} )?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[,.]? \d{4}'


# Caso 6: Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009

# Una letra mayúscula, seguida de una o más letras minúsculas, seguida de un espacio en blanco, 
# seguida de un número que tiene entre una o dos cifras, seguida de una o más letras
# minúsculas, seguida de un espacio en blanco, seguida de un número que tiene entre dos y cuatro cifras

caso6 = '[A-Z][a-z]+ \d{1,2}[a-z]+ \d{2,4}'

# Caso 7: 2009; 2010

# un número 1 o un número 2 (Porque puede que haya un numero como 3007 pero es fecha aún no ocurre) 
# seguido de un número de 3 cifras (Para completar los miles, es decir, los números con cuatro cifras)
caso7 = '([1|2]\d{3})'


In [3]:
# guardo en una lista cada línea del archivo txt (cada elemento de la lista es un renglón)
archivo = []
with open('dates.txt') as doc:
    for linea in doc:
        archivo.append(linea)
# creo un series de pandas , donde cada linea del txt es un renglon
df = pd.Series(archivo)

# aplico las expresiones regulares que hice para encontrarlas 
# De todos los casos que ya encontré, ahora voy a aplicar una conjunción excluyente con el símbolo '|'

fechas_ordenadas = df.str.extract('('+caso1+'|'+caso2+'|'+caso3+'|'+caso4+'|'+caso5+'|'+caso6+'|'+caso7+')')


# nos quedamos con la primera columna 
fechas_ordenadas = fechas_ordenadas.iloc[:,0]
# convertimos todo a fecha 
fechas_ordenadas = pd.Series(pd.to_datetime(fechas_ordenadas))
# ordenamos todo ascendentemente
fechas_ordenadas = fechas_ordenadas.sort_values(ascending=True)

# mostramos el resultado del ordenamiento de las fechas
fechas_ordenadas


9     1971-04-10
84    1971-05-18
2     1971-07-08
53    1971-07-11
28    1971-09-12
474   1972-01-01
153   1972-01-13
13    1972-01-26
129   1972-05-06
98    1972-05-13
111   1972-06-10
225   1972-06-15
31    1972-07-20
171   1972-10-04
191   1972-11-30
486   1973-01-01
335   1973-02-01
415   1973-02-01
36    1973-02-14
405   1973-03-01
323   1973-03-01
422   1973-04-01
375   1973-06-01
380   1973-07-01
345   1973-10-01
57    1973-12-01
481   1974-01-01
436   1974-02-01
104   1974-02-24
299   1974-03-01
162   1974-03-05
154   1974-03-06
402   1974-04-01
95    1974-04-11
73    1974-04-12
108   1974-05-04
156   1974-05-26
332   1974-06-01
182   1974-06-13
82    1974-07-07
351   1974-08-01
278   1974-09-01
214   1974-09-10
155   1974-10-10
223   1974-10-14
473   1975-01-01
49    1975-02-28
317   1975-03-01
11    1975-04-09
319   1975-07-01
40    1975-07-29
418   1975-08-01
165   1975-08-18
370   1975-09-01
382   1975-09-01
3     1975-09-27
50    1975-11-22
363   1975-12-01
219   1975-12-

In [4]:
lista_fechas_ordenadas = list(fechas_ordenadas)

In [5]:
lista_fechas_ordenadas

[Timestamp('1971-04-10 00:00:00'),
 Timestamp('1971-05-18 00:00:00'),
 Timestamp('1971-07-08 00:00:00'),
 Timestamp('1971-07-11 00:00:00'),
 Timestamp('1971-09-12 00:00:00'),
 Timestamp('1972-01-01 00:00:00'),
 Timestamp('1972-01-13 00:00:00'),
 Timestamp('1972-01-26 00:00:00'),
 Timestamp('1972-05-06 00:00:00'),
 Timestamp('1972-05-13 00:00:00'),
 Timestamp('1972-06-10 00:00:00'),
 Timestamp('1972-06-15 00:00:00'),
 Timestamp('1972-07-20 00:00:00'),
 Timestamp('1972-10-04 00:00:00'),
 Timestamp('1972-11-30 00:00:00'),
 Timestamp('1973-01-01 00:00:00'),
 Timestamp('1973-02-01 00:00:00'),
 Timestamp('1973-02-01 00:00:00'),
 Timestamp('1973-02-14 00:00:00'),
 Timestamp('1973-03-01 00:00:00'),
 Timestamp('1973-03-01 00:00:00'),
 Timestamp('1973-04-01 00:00:00'),
 Timestamp('1973-06-01 00:00:00'),
 Timestamp('1973-07-01 00:00:00'),
 Timestamp('1973-10-01 00:00:00'),
 Timestamp('1973-12-01 00:00:00'),
 Timestamp('1974-01-01 00:00:00'),
 Timestamp('1974-02-01 00:00:00'),
 Timestamp('1974-02-

In [6]:

# esta lista ya no viene como timestamp (cada fecha como si fuera un string)
# por si lo quieren revisar así. Los acomodé en el orden Año/Mes/día
# Y = Year
# m = month
# d = day
# si quieren modificar el orden en el que se pone la fecha para facilitar la 
# revisión, basta con indicarlo en el argumento de el método "strftime"
# esto lo comento porque creo que en las instrucciones no viene especificado
# cómo tiene que ir el orden de los elementos de la lista 
lista_fechas_ordenadas_no_timestamp = []

for fecha in lista_fechas_ordenadas:
    lista_fechas_ordenadas_no_timestamp.append(fecha.strftime('%Y/%m/%d'))


In [7]:
lista_fechas_ordenadas_no_timestamp

['1971/04/10',
 '1971/05/18',
 '1971/07/08',
 '1971/07/11',
 '1971/09/12',
 '1972/01/01',
 '1972/01/13',
 '1972/01/26',
 '1972/05/06',
 '1972/05/13',
 '1972/06/10',
 '1972/06/15',
 '1972/07/20',
 '1972/10/04',
 '1972/11/30',
 '1973/01/01',
 '1973/02/01',
 '1973/02/01',
 '1973/02/14',
 '1973/03/01',
 '1973/03/01',
 '1973/04/01',
 '1973/06/01',
 '1973/07/01',
 '1973/10/01',
 '1973/12/01',
 '1974/01/01',
 '1974/02/01',
 '1974/02/24',
 '1974/03/01',
 '1974/03/05',
 '1974/03/06',
 '1974/04/01',
 '1974/04/11',
 '1974/04/12',
 '1974/05/04',
 '1974/05/26',
 '1974/06/01',
 '1974/06/13',
 '1974/07/07',
 '1974/08/01',
 '1974/09/01',
 '1974/09/10',
 '1974/10/10',
 '1974/10/14',
 '1975/01/01',
 '1975/02/28',
 '1975/03/01',
 '1975/04/09',
 '1975/07/01',
 '1975/07/29',
 '1975/08/01',
 '1975/08/18',
 '1975/09/01',
 '1975/09/01',
 '1975/09/27',
 '1975/11/22',
 '1975/12/01',
 '1975/12/14',
 '1976/01/01',
 '1976/02/01',
 '1976/02/11',
 '1976/03/01',
 '1976/03/10',
 '1976/04/01',
 '1976/06/01',
 '1976/09/

In [8]:
# por si se les facilita revisar con el índice

indice_fechas_ordenadas = fechas_ordenadas.index
indice_fechas_ordenadas

Int64Index([  9,  84,   2,  53,  28, 474, 153,  13, 129,  98,
            ...
            152, 235, 464, 253, 427, 231, 141, 186, 161, 413],
           dtype='int64', length=500)

In [9]:
indice_fechas_ordenadas.values

array([  9,  84,   2,  53,  28, 474, 153,  13, 129,  98, 111, 225,  31,
       171, 191, 486, 335, 415,  36, 405, 323, 422, 375, 380, 345,  57,
       481, 436, 104, 299, 162, 154, 402,  95,  73, 108, 156, 332, 182,
        82, 351, 278, 214, 155, 223, 473,  49, 317,  11, 319,  40, 418,
       165, 370, 382,   3,  50, 363, 219, 465, 237,  23, 342, 204, 258,
       315,  27,  93,  17, 488, 303, 283, 395, 309, 419, 123,  19, 117,
       232,  72, 189, 369, 493, 318, 239, 148, 105, 336,   6, 200,  81,
        65, 434, 164, 378, 313, 495, 424, 398,   5, 254, 296,  75, 167,
        21, 259, 499, 347, 150,  78, 340, 441, 361, 267, 221, 466,  39,
       134, 197, 355, 430,  80, 444, 246,  85, 215, 263,  74, 403, 458,
        16,  25, 127, 454,  70,  44,  59, 103, 112, 429,  88, 179, 470,
       358, 205, 397, 294, 137, 295,  35, 438, 247, 209,  61, 107, 285,
       175,  99, 455,  24, 275, 421,  48, 426, 489, 136,  30, 274,  10,
       178,   1, 447, 280, 185, 228, 135,  69, 492, 199, 352,   